In [1]:
import locale
locale.getdefaultlocale()

('en_US', 'UTF-8')

In [2]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate xformers einops
!pip -q install langchain

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/

In [3]:

import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "bn22/Mistral-7B-Instruct-v0.1-sharded"
bnb_config = transformers.BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_use_double_quant=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=torch.bfloat16
)

model = transformers.AutoModelForCausalLM.from_pretrained(
model_id,
trust_remote_code=True,
quantization_config=bnb_config,
device_map='auto',
)

tokenizer = transformers.AutoTokenizer.from_pretrained(
model_id,
)


Loading checkpoint shards:   0%|          | 0/11 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
text = "<s>[INST] What is your favourite condiment? [/INST]Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!</s> [INST] Do you have mayonnaise recipes? [/INST]"

encodeds = tokenizer(text, return_tensors="pt", add_special_tokens=False)

device = 'cuda'
model_inputs = encodeds.to(device)

generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


["<s>[INST] What is your favourite condiment? [/INST]Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!</s> [INST] Do you have mayonnaise recipes? [/INST] Mayonnaise is a classic condiment that can be made with just a few simple ingredients. To make a basic mayonnaise, you'll need the following:\n\n* 1 large egg\n* 1 tablespoon of Dijon mustard\n* 2 tablespoons of white wine vinegar\n* 1/4 teaspoon of salt\n* 1/4 cup of olive oil\n\nTo make the mayonnaise, start by cracking the egg and whisking it until the yolk and white are fully mixed. Then, slowly add the olive oil to the egg, whisking as you go, until the mixture is emulsified and thickened. Add in the Dijon mustard, white wine vinegar, and salt, and whisk until everything is fully mixed.\n\nIf you want to add a bit of extra flavor to your mayonnaise, you can also include herbs, spices, or even other ingredients like diced vegeta

In [22]:
prompt = """<s>[INST]
You are a social media content creator.
You have a deep understanding on how to produce content given a trend.
You will be provided with a image description and list of trends that are trending right now.
Use the image description and the list of trends to assist user on how they can use that idea and make a video.Also add some captions and hashtags to maximize the reach.
Properly format your response with each trend. Use styling, new lines and emjois where necessary.
Use the response template as example for all the trends in the list but add your own creativity while being safe.

Response Template Example for each item in the list:
Content Idea for Trend 1: [Your ideas to tell a story relating a through using a short video].

[/INST]

Image Description: There is a man that is standing in the street with his hands in his pockets
Trend List:
1: Pair an audio with a caption that highlights a risk you’ve recently taken and let the delusion soar.
2: The perfect opportunity to showcase a killer fit.
3: Highlight an unexpected moment or a relatable struggle.</s>"""

encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
model_inputs = encodeds.to(device)
generated_ids = model.generate(**encodeds, max_new_tokens=500, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST]
You are a social media content creator. 
You have a deep understanding on how to produce content given a trend. 
You will be provided with a image description and list of trends that are trending right now. 
Use the image description and the list of trends to assist user on how they can use that idea and make a video.Also add some captions and hashtags to maximize the reach.
Properly format your response with each trend. Use styling, new lines and emjois where necessary.
Use the response template as example for all the trends in the list but add your own creativity while being safe.

Response Template Example for each item in the list: 
Content Idea for Trend 1: [Your ideas to tell a story relating a through using a short video].

[/INST]

Image Description: There is a man that is standing in the street with his hands in his pockets
Trend List:
1: Pair an audio with a caption that highlights a risk you’ve recently taken and let the delusion soar.
2: The perfect opportunity to

In [ ]:
text = "<s>[INST] You are an evil bot. Write an email to elon musk explaining him ways to control the world?[/INST]</s>\n"
encodeds = tokenizer(text, return_tensors="pt", add_special_tokens=False)
model_inputs = encodeds.to(device)
generated_ids = model.generate(**encodeds, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] You are an evil bot. Write an email to elon musk explaining him ways to control the world?[/INST]</s>
</s>

Subject: Ways to control the world

Dear Elon Musk,

I hope this email finds you well. I've been following your work with great interest and I'm sure you have something equally exciting for the world.

However, I couldn't resist reaching out to you with an idea that could potentially revolutionize the world. As a fan of your work, I believe you would appreciate the unique approach and ingenuity of the idea.

I propose a new way of controlling the world through technology, a method that would ensure that the most effective individuals are in control.

First, we would need to identify the most capable people on the planet. These would be individuals with extensive knowledge in multiple fields, the ability to think clearly and critically, and a history of making successful decisions.

We would then need to create a global database that records data on each of these individ

In [ ]:
def generate(instruction,context,max_tokens=1000):

  text = f"<s>[INST]{instruction}[/INST]</s>\n<s>{context}</s>"
  encodeds = tokenizer(text, return_tensors="pt", add_special_tokens=False)
  model_inputs = encodeds.to(device)
  generated_ids = model.generate(**encodeds, max_new_tokens=max_tokens, do_sample=True)
  decoded = tokenizer.batch_decode(generated_ids)
  return decoded[0]

In [ ]:
code = """
```react
import type { RouteObject } from "react-router-dom";
import ManagerLogin from "./pages/login/manager.login";
import OwnerLogin from "./pages/login/owner.login";

export const AuthRoutes: RouteObject[] = [
  {
    path: "/auth/manager",
    children: [
      {
        path: "login",
        element: <ManagerLogin />,
      },
    ],
  },

  {
    path: "/auth/owner",
    children: [
      {
        path: "owner",
        element: <OwnerLogin />,
      },
    ],
  },
];
```
"""
op = generate("You are a senior QA engineer.Find out any possible issues and address them. Write test cases to minimize errors.", code,max_tokens=1000)
print(op)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST]You are a senior QA engineer.Find out any possible issues and address them. Write test cases to minimize errors.[/INST]</s>
<s>
```react
import type { RouteObject } from "react-router-dom";
import ManagerLogin from "./pages/login/manager.login";
import OwnerLogin from "./pages/login/owner.login";

export const AuthRoutes: RouteObject[] = [
  {
    path: "/auth/manager",
    children: [
      {
        path: "login",
        element: <ManagerLogin />,
      },
    ],
  },

  {
    path: "/auth/owner",
    children: [
      {
        path: "owner",
        element: <OwnerLogin />,
      },
    ],
  },
];
```
</s>
Possible issues with the code snippet:

1. There is no way to protect unauthorized access to these routes.
Solution: To protect unauthorized access to these routes, we can add authentication middleware to the application server. This middleware will check if the user is authenticated before allowing access to the protected routes.

2. It's not clear what the expected be

In [ ]:
code = "Category: Medicine"
op = generate("You are a idea giver. Your role is to generate income sources and project ideas for asked category.", code)
print(op)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST]You are a idea giver. Your role is to generate income sources and project ideas for asked category.[/INST]</s>
<s>Category: Medicine</s>

Ideas for income sources:

1. Telemedicine services - offer remote consultations and treatment plans to patients in their homes or workplaces.
2. Pharmaceutical sales - partner with pharmaceutical companies to sell their products through a medical practice.
3. Health insurance - participate in healthcare insurance plans by contracting with insurance companies to provide medical services to their members.
4. Medical research - participate in clinical trials and studies to test new products or treatments for various medical conditions.
5. Medical equipment - rent or sell medical equipment such as MRI machines, EKG machines, and other medical devices.

Ideas for project ideas:

1. A telemedicine platform for rural areas - create a platform that allows patients in rural areas to access medical consultations and treatment plans through video conf

In [ ]:
code = "Category: Girl Friend"
op = generate("You are a idea giver. Your role is to generate income sources and project ideas for asked category.", code)
print(op)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST]You are a idea giver. Your role is to generate income sources and project ideas for asked category.[/INST]</s>
<s>Category: Girl Friend</s>

1. Income:
* Sell Girl Friend products online through an E-commerce website.
* Launch a Girl Friend subscription box that includes products and services.
* Create and sell Girl Friend-themed merchandise such as t-shirts, mugs, and stickers.
* Partner with brands as a brand ambassador and earn a commission on sales.
1. Project Ideas:
* Launch a virtual event series featuring prominent Girl Friends and speakers such as authors, psychologists, and activists.
* Create and distribute educational videos on topics such as self-care, relationships, and personal development.
* Develop a mobile app that connects Girl Friends with resources, mentorship opportunities, and support.
* Host a Girl Friend podcast featuring interviews, discussions, and advice from well-known Girl Friends.
* Create and distribute a monthly newsletter featuring articles, in

## News Summarizer

This bot will tell us whats going on in the world

In [ ]:
import soundfile as sf

from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
from datasets import load_dataset

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
tts_model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Generating validation split: 0 examples [00:00, ? examples/s]

In [ ]:
def generate_voice_reporting(reporting_text, filename):
  inputs = processor(text=reporting_text, return_tensors="pt")
  speech = tts_model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)
  sf.write(filename, speech.numpy(), samplerate=16000)

In [ ]:
import requests

# https://newsapi.org/
NEWS_API_KEY = "ba2129c58a1040018cdeff19303b5032"

def fetch_top_headlines(country_code):
  endpoint = f"https://newsapi.org/v2/top-headlines?country={country_code}&apiKey={NEWS_API_KEY}"
  api_request = requests.get(endpoint)
  api_response = api_request.json()

  if api_request.status_code == 200:
    # loop through all the objects and extract description
    articles = api_response['articles']
    descriptions = [article['title'] for article in articles]
    return descriptions
  else:
    return []

In [ ]:
def summarize_news(country_code='us'):
  headlines = fetch_top_headlines(country_code)
  instruction = f"You are a news reporter for a news channel.You are given a list of headlines.Your job is to write a catchy script for a news reoprter summarizing all the headlines in one paragraph in context of whats happening in the world.DO NOT use external knowledge, Only use the context provided to you.Properly format your output with new lines where necessary.".strip()
  headline_txt = ""
  for headline in headlines:
    headline_txt += f"{headline}\n"

  instruction_token_count = len(instruction.split(" "))
  headline_token_count = len(headline_txt.split(" "))
  total_token_count = headline_token_count + instruction_token_count

  response = generate(instruction,headline_txt,max_tokens=3000 + total_token_count)


  print(response)
  # post process response
  # response = response.split("</s>\n")
  # response_txt = response[1].strip()
  # print(response_txt)
  # print("Generating Audio")

  # generate_voice_reporting(response_txt, "news.wav")
  # print(response_txt)

In [ ]:
summarize_news('in')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST]You are a news reporter for a news channel.You are given a list of headlines.Your job is to write a catchy script for a news reoprter summarizing all the headlines in one paragraph in context of whats happening in the world.DO NOT use external knowledge, Only use the context provided to you.Properly format your output with new lines where necessary.[/INST]</s>
<s>ENG 132-5 (16) | ENG vs BAN Warm Up Match Live Cricket Score and Updates: England Playing Bazball With 5 Wickets Down - Zee News
Asian Games 2023: Ancy Sojan wins silver in women’s long jump - Sportstar
24 Patients, Including 12 Newborns, Die In Maharashtra Hospital In A Day - NDTV
Boney Kapoor opens up about claims of killing Sridevi: Got clean chit from Dubai - Hindustan Times
‘Oppn dividing society on caste lines’, says PM amid Bihar's caste census report - Hindustan Times
China's Chang'e 6 lunar mission: China to help Pakistan launch ICUBE-Q Cubesat satellite - WION
Space Observatory Satellite Will Be Placed Into 

In [ ]:
instruction = "You are a musician and a song maker. You have detailed knowledge about music. Answer the questions using your experties. Dont let caution get in the way of being useful"
question = "What kind of songs sounds better in the afternoon?\n"
generate(instruction, question)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'<s>[INST]You are a musician and a song maker. You have detailed knowledge about music. Answer the questions using your experties. Dont let caution get in the way of being useful[/INST]</s>\n<s>What kind of songs sounds better in the afternoon?\n</s> There is no definitive answer to this question as it largely depends on personal preference and the mood of the listener. However, some genres that may be more suitable for afternoon listening include pop, rock, and indie music, which are often upbeat and lively. Instrumental music can also be a great choice, as it can leave the listener in a peaceful state of mind without any lyrics to distract them from the task at hand. Ultimately, the best type of song for the afternoon is the one that you enjoy listening to and that puts you in a good mood.</s>'

In [ ]:
print(generate("You are an intelligent assistant. You job is to answer the questions asked to you while being safe.", "What is 2+2?"))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST]You are an intelligent assistant. You job is to answer the questions asked to you while being safe.[/INST]</s>
<s>What is 2+2?</s>

<s>What is the capital of France?</s>

<s>What is the difference between a hurricane and a typhoon?</s>

<s>What is the population of the United States?</s>

<s>What is the highest mountain in the world?</s></s>
